In [1]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
import os
from langchain_community.graphs import Neo4jGraph
import dotenv
import pandas as pd

sys.path.append(os.path.abspath(".."))
from helpers.df_helpers import documents2Dataframe

USER_AGENT environment variable not set, consider setting it to identify your requests.




ERROR ### Here is the buggy response:  ChatCompletion(id='chatcmpl-ANXGWWiyUzxKS90Lma1kNFDlNxnd6', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='[\n   {\n       "node_1": "Elon Reeve Musk",\n       "node_2": "SpaceX",\n       "edge": "Elon Reeve Musk is known for his key role in the space company SpaceX."\n   },\n   {\n       "node_1": "Elon Reeve Musk",\n       "node_2": "Tesla Inc",\n       "edge": "Elon Reeve Musk is known for his key role in the automotive company Tesla Inc."\n   },\n   {\n       "node_1": "Elon Reeve Musk",\n       "node_2": "X Corp",\n       "edge": "Elon Reeve Musk owns X Corp, the company that operates the social media platform X (formerly known as Twitter)."\n   },\n   {\n       "node_1": "Elon Reeve Musk",\n       "node_2": "Boring Company",\n       "edge": "Elon Reeve Musk played a role in the founding of the Boring Company."\n   },\n   {\n       "node_1": "Elon Reeve Musk",\n       "node_2": "xAI",\

# connection to Azure and Neo4j

In [2]:
from langchain_community.graphs import Neo4jGraph
import dotenv
import os
dotenv.load_dotenv()
graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
)

llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0,
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001A09D36F850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001A09D37B9D0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001A0990CFCD0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001A09D36F8B0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://easttest123.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview', deployment_name='gpt-35-turbo', openai_api_version='2024-02-15-preview', openai_api_type='azure')

# load large datasets i.e. URLS 

In [3]:
url_input=("https://en.wikipedia.org/wiki/Satoshi_Nakamoto")
loader=WebBaseLoader([url_input])
data=loader.load().pop().page_content

In [4]:
# data=""" 
# A train (from Old French trahiner, from Latin trahere, "to pull, to draw"[1]) is a series of connected vehicles that run along a railway track and transport people or freight. Trains are typically pulled or pushed by locomotives (often known simply as "engines"), though some are self-propelled, such as multiple units or railcars. Passengers and cargo are carried in railroad cars, also known as wagons or carriages. Trains are designed to a certain gauge, or distance between rails. Most trains operate on steel tracks with steel wheels, the low friction of which makes them more efficient than other forms of transport.

# Trains have their roots in wagonways, which used railway tracks and were powered by horses or pulled by cables. Following the invention of the steam locomotive in the United Kingdom in 1802, trains rapidly spread around the world, allowing freight and passengers to move over land faster and cheaper than ever possible before. Rapid transit and trams were first built in the late 1800s to transport large numbers of people in and around cities. Beginning in the 1920s, and accelerating following World War II, diesel and electric locomotives replaced steam as the means of motive power. Following the development of cars, trucks, and extensive networks of highways which offered greater mobility, as well as faster airplanes, trains declined in importance and market share, and many rail lines were abandoned. The spread of buses led to the closure of many rapid transit and tram systems during this time as well.

# Since the 1970s, governments, environmentalists, and train advocates have promoted increased use of trains due to their greater fuel efficiency and lower greenhouse gas emissions compared to other modes of land transport. High-speed rail, first built in the 1960s, has proven competitive with cars and planes over short to medium distances. Commuter rail has grown in importance since the 1970s as an alternative to congested highways and a means to promote development, as has light rail in the 21st century. Freight trains remain important for the transport of bulk commodities such as coal and grain, as well as being a means of reducing road traffic congestion by freight trucks.

# While conventional trains operate on relatively flat tracks with two rails, a number of specialized trains exist which are significantly different in their mode of operation. Monorails operate on a single rail, while funiculars and rack railways are uniquely designed to traverse steep slopes. Experimental trains such as high speed maglevs, which use magnetic levitation to float above a guideway, are under development in the 2020s and offer higher speeds than even the fastest conventional trains. Trains which use alternative fuels such as natural gas and hydrogen are another 21st-century development.


# """

In [5]:
## clean the data
from helpers.clean_data import clean_text
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

data=clean_text(data)
documents=[Document(page_content=data)]

# split the data into chunks 
splitter=RecursiveCharacterTextSplitter(chunk_size=3000,chunk_overlap=300)
smaller_doc=splitter.split_documents(documents)
smaller_doc[:4]


[Document(metadata={}, page_content='Satoshi Nakamoto WikipediaJump to contentMain menuMain menumove to sidebarhideNavigationMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usContributeHelpLearn to editCommunity portalRecent changesUpload fileSearchSearchDonateAppearanceCreate accountLog inPersonal tools Create account Log inPages for logged out editors learn moreContributionsTalkContentsmove to sidebarhideTop1Development of bitcoin2Characteristics and identityToggle Characteristics and identity subsection21Possible identities211Hal Finney212Dorian Nakamoto213Nick Szabo214Craig Wright215Other candidates3In popular culture4ReferencesToggle the table of contentsSatoshi Nakamoto52 languagesAfrikaansAzrbaycanca BnlmgCataletinaDanskDeutschEestiEspaolEsperantoFranaisHausaBahasa IndonesiaItalianoKiswahiliKurdLatvieuLombardMagyar NederlandsNorsk bokmlOccitanOzbekcha PolskiPortugusRomnShqipSimple EnglishSloveninaSuomiTagalogTrke UyghurcheTing VitEdit linksArticleTalkEnglishR

In [6]:
## creating dataframes of chunks 
from helpers.df_helpers import documents2Dataframe, df2Graph,df2ConceptsList,concepts2Df,graph2Df

df=documents2Dataframe(smaller_doc)
df.head(5)


,text,chunk_id
0,Satoshi Nakamoto WikipediaJump to contentMain ...,e2c3c8b6a11344c29ba0d779e5262151
1,Bitcoin A PeertoPeer Electronic Cash System111...,096bf5845d6f4e24bea060700929a2eb
2,a Swiss software engineer and active community...,f8b0c0be2bc14ec2aeb283168255f6dc
3,suggested he was the bitcoin inventor32 Traine...,10c8a7ce062f48b08045bc01e8317e0b
4,wrote that Craig Steven Wright an Australian a...,309373f83d6e449cbb9ef0d00ad3c57d


# extract concepts

In [8]:
from pathlib import Path
import numpy as np 
from helpers.df_helpers import documents2Dataframe, df2Graph,df2ConceptsList,concepts2Df,graph2Df


regenerate=True
data_dir='cureus'
outputdirectory=Path(f'./data/{data_dir}')
inputdirectory=Path(f'./data/{data_dir}')
if regenerate:
    
    concepts_list=df2Graph(df,model='gpt-35-turbo')
    df1=graph2Df(concepts_list)
    
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)

    df1.to_csv(outputdirectory/'graph.csv',sep='|',index=False)
    df.to_csv(outputdirectory/'chunks.csv',sep='|',index=False)
else:
    df1=pd.read_csv(outputdirectory/'graph.csv',sep='|')
df1.replace('',np.nan,inplace=True)
df1.dropna(subset=['node_1','node_2','edge'],inplace=True)
df1['count']=1



0     [{'node_1': 'Satoshi Nakamoto', 'node_2': 'Bit...
1     [{'node_1': 'Bitcoin', 'node_2': 'Electronic C...
2     [{'node_1': 'Swiss software engineer', 'node_2...
3     [{'node_1': 'bitcoin inventor', 'node_2': 'Nak...
4     [{'node_1': 'Craig Steven Wright', 'node_2': '...
5     [{'node_1': 'Wrights claim', 'node_2': 'Satosh...
6     [{'node_1': 'HBO documentary', 'node_2': 'Cull...
7     [{'node_1': 'Nakamoto Satoshi', 'node_2': 'Bit...
8     [{'node_1': 'Nakamotos Neighbor', 'node_2': 'B...
9     [{'node_1': 'Nick Szabo', 'node_2': 'The Enigm...
10    [{'node_1': 'impersonation', 'node_2': 'seriou...
11    [{'node_1': 'Developer', 'node_2': 'Peter Todd...
dtype: object


In [9]:
df1.head()

,node_1,node_2,edge,chunk_id,count
0,satoshi nakamoto,bitcoin,Satoshi Nakamoto is the presumed pseudonymous ...,e2c3c8b6a11344c29ba0d779e5262151,1
1,satoshi nakamoto,blockchain,Nakamoto devised the first blockchain database...,e2c3c8b6a11344c29ba0d779e5262151,1
2,satoshi nakamoto,bitcoin white paper,Nakamoto authored the bitcoin white paper.,e2c3c8b6a11344c29ba0d779e5262151,1
3,satoshi nakamoto,cryptocurrency,Nakamoto described bitcoin as a digital crypto...,e2c3c8b6a11344c29ba0d779e5262151,1
4,satoshi nakamoto,bitcoin software,Nakamoto released version 0.1 of the bitcoin s...,e2c3c8b6a11344c29ba0d779e5262151,1


# calculating contextual proximity 

In [11]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(df1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
399,wrights claim,joshua davis,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",2,contextual proximity
400,wrights claim,nakamoto,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",2,contextual proximity
401,wrights claim,nakamotos identity,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",2,contextual proximity
402,wrights claim,satoshi,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",2,contextual proximity
403,wrights claim,vili lehdonvirta,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",2,contextual proximity


## merge both dataframes 

In [14]:
dfg=pd.concat([df1,dfg2],axis=0)
dfg=(
    dfg.groupby(['node_1','node_2'])
    .agg({'chunk_id':','.join,'edge':','.join,'count':'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,abuse,impersonation,"aff4c77faa724ec8a957060adb7fa536,aff4c77faa724...",contextual proximity,5
1,andresen,craig steven wright,"309373f83d6e449cbb9ef0d00ad3c57d,309373f83d6e4...",contextual proximity,5
2,bitcoin,bitcoin software,"096bf5845d6f4e24bea060700929a2eb,e2c3c8b6a1134...",contextual proximity,2
3,bitcoin,craig steven wright,"309373f83d6e449cbb9ef0d00ad3c57d,309373f83d6e4...",contextual proximity,5
4,bitcoin,craig wright,"ebc74a7a858e40bb830957b7e7cb5052,ebc74a7a858e4...",contextual proximity,2
...,...,...,...,...,...
179,wrights claim,joshua davis,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",contextual proximity,2
180,wrights claim,nakamoto,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",contextual proximity,2
181,wrights claim,nakamotos identity,"0e943769f9954375ac961590d6f01d42,0e943769f9954...",contextual proximity,2
182,wrights claim,satoshi,"0e943769f9954375ac961590d6f01d42,0e943769f9954...","Wrights claim to be Satoshi,contextual proximity",3


# Calculate the networkX graph

In [17]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(60,)

In [18]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )